# Predicting Impact of Forest Fires

## Introduction

## Exploratory Data Analysis

## Methods: Plan

In this analysis, we are drawing from a data set that is trustworthy. For one, there are ___ missing values. In addition, the nature of the data is objective. By taking measurements of location, weather aspects, and area burnt, there is very little human subjectivity involved. Finally, by setting the seed and explaining our decision-making along the way, we produce a reproducible and trustworthy report.

The following is our plan for the rest of the report:

- With our training data, use forward subsitution to choose predictor variables
- With our training data, use Asymptotic and Bootstrap methods to create a linear regression model with "area burned" as our response variable 
- Compare the Asymptotic and Bootstrap methods
- Check for problems such as Heteroscedasticity, Normality, and  Multicollinearity in our model, then adjust our model as
- With the test data, evaluate our statistical model using RMSPE
- Make a prediction using local forest data from British Columbia, while keeping in mind issues around generalizability 

By the end of our final report we expect to be able to predict with reasonable accuracy the area burned in a forest fire in Portugal. We also expect to go through a systematic, reproducible, process to determine which predictor variables create a good model.

Our report could be useful in many ways. For one, our results could help with resource designation. We only have finite resources for fire-fighting. Therfore, if we know that low humidity is a big indicator of forset fire area, we can set up low-humidity areas with more fire-fighting resources than high-humidity areas. In addition, our results could help with fire-preperation. If data indicates that if a fire were to occur, it would cover a very large area, it could be prudent to alert citizens of this fire danger.

## References